In [1]:
from pymongo import MongoClient
import configparser
import pandas as pd
import re
from nltk.corpus import wordnet
import numpy as np
from flask import Flask, render_template
# import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df = px.data.wind()
fig = px.scatter_polar(df, r="strength", theta="direction", color="frequency",
                       color_continuous_scale='Reds',
                       color_continuous_midpoint=np.median(df['frequency']),
                       range_color=[df['frequency'].min(), df['frequency'].max()])
fig.show()

### 資料處理

In [21]:
_df = pd.read_csv("./data.csv")

In [22]:
# 建立字典，將年月份對應到角度
month_labels = ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
               '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01',
               '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01']
angle_mapping = [(i * 15) % 360 for i in range(len(month_labels))]
dict_date_angle = dict(zip(month_labels, angle_mapping))


In [4]:
def transform(text):
    text = str(text)
    year = text[:4]
    month = text[-2:]
    return year + "-" + month

In [14]:
# 將日期轉換為角度
def date_to_angle(date):
    date = date.strftime("%Y-%m-%d")
    return dict_date_angle[date]

In [23]:
_df['monitormonth'] = pd.to_datetime(_df["monitormonth"].apply(lambda x: transform(x)), format='%Y-%m')

In [16]:
_df["monitormonth"] = _df["monitormonth"].apply(lambda x: transform(x))

In [17]:
_df['monitormonth'] = pd.to_datetime(_df['monitormonth'], format='%Y-%m')

In [ ]:
# find the min of monitormonth
min_date = _df['monitormonth'].min()
max_date = _df['monitormonth'].max()
print(min_date.strftime('%Y-%m-%d'), "~", max_date.strftime('%Y-%m-%d'))

In [24]:
start_date = pd.to_datetime('2021-01', format='%Y-%m')
end_date = pd.to_datetime('2022-12', format='%Y-%m')
filtered_df = _df[(_df['monitormonth'] >= start_date) & (_df['monitormonth'] <= end_date)]

In [25]:
loc = filtered_df[filtered_df['sitename']=="屏東"]
loc = pd.pivot_table(loc, values='concentration', index=loc['monitormonth'], columns='itemname')
loc = loc.reset_index()
df_location = loc[['monitormonth', '細懸浮微粒', '懸浮微粒', '一氧化氮', '相對濕度']]
df_location['angle'] = df_location.monitormonth.apply(lambda x: date_to_angle(x))

c:\Users\User\anaconda3\envs\text\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
df_location

itemname,monitormonth,細懸浮微粒,懸浮微粒,一氧化氮,相對濕度,angle
0,2021-01-01,36.0,61.0,2.5,68.0,0
1,2021-02-01,33.0,57.0,1.7,68.0,15
2,2021-03-01,32.0,62.0,2.2,68.0,30
3,2021-04-01,23.0,53.0,2.1,71.0,45
4,2021-05-01,11.0,30.0,1.7,68.0,60
5,2021-06-01,6.0,17.0,1.6,80.0,75
6,2021-07-01,8.0,19.0,1.1,79.0,90
7,2021-08-01,9.0,19.0,1.3,82.0,105
8,2021-09-01,15.0,28.0,1.0,77.0,120
9,2021-10-01,19.0,38.0,1.1,76.0,135


In [13]:
import plotly.graph_objects as go

# 生成示例數據
# r = [1, 2, 3, 4, 3, 2, 1, 2]

# 自定義的年月份標籤
month_labels = ['2021/01', '2021/02', '2021/03', '2021/04', '2021/05', '2021/06',
               '2021/07', '2021/08', '2021/09', '2021/10', '2021/11', '2021/12',
               '2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
               '2022/07', '2022/08', '2022/09', '2022/10', '2022/11', '2022/12']

# 創建 Polar Charts
fig = go.Figure()

# 添加數據到圖表，並設定標籤文本
fig.add_trace(go.Scatterpolar(
    name='相對濕度',
    r=df_location['相對濕度'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['相對濕度'].tolist(),  # 根據 r 值設定顏色
        colorscale='Blues',  # 設定顏色漸層
        # colorbar=dict(
        #     title='相對濕度'  # 設定顏色軸標題
        # ),
        cmin=0.9*min(df_location['相對濕度'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['相對濕度'].tolist())  # 設定顏色漸層最大值
    ),
    
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>percent: %{r} %<br>",  # 定義 hover 顯示的文本格式
))
fig.add_trace(go.Scatterpolar(
    name='細懸浮微粒',
    r=df_location['細懸浮微粒'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['細懸浮微粒'].tolist(),  # 根據 r 值設定顏色
        colorscale='Reds',  # 設定顏色漸層
        # colorbar=dict(
        #     title='細懸浮微粒'  # 設定顏色軸標題
        # ),
        cmin=0.9*min(df_location['細懸浮微粒'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['細懸浮微粒'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式
))
fig.add_trace(go.Scatterpolar(
    name='懸浮微粒',
    r=df_location['懸浮微粒'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['懸浮微粒'].tolist(),  # 根據 r 值設定顏色
        colorscale='Greens',  # 設定顏色漸層
        # colorbar=dict(
        #     title='懸浮微粒'  # 設定顏色軸標題
        # ),
        cmin=min(df_location['懸浮微粒'].tolist()),  # 設定顏色漸層最小值
        cmax=0.9*max(df_location['懸浮微粒'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式 
))

# 設定角度刻度和標籤，並且顏色填充用藍色根據r值由小到大漸層
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            angle=0,
            tickvals=angle_mapping,
            ticks="outside",
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    ),
    height=600,
    # width=600,
    # showlegend=False
)

# 顯示圖表
fig.show()

In [ ]:
import plotly.graph_objects as go

# 設定角度刻度和標籤
angles = [i * 15 for i in range(24)]  # 每十五度一個刻度
labels = [str(i) + "°" for i in angles]  # 刻度標籤

# 創建 Polar Chart
fig = go.Figure()

# 添加從中心往外的線
for angle in angles:
    fig.add_trace(go.Scatterpolar(
        r=[0, 1],  # 從中心 (r=0) 繪製到半徑為 1 的線
        theta=[angle, angle],  # 角度起點和終點相同，畫一條直線
        mode='lines',
        line=dict(
            color='black',
            width=1
        ),
        hoverinfo='skip'  # 不顯示 hover 資訊
    ))

# 設定角度刻度和標籤
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False  # 隱藏半徑軸
        ),
        angularaxis=dict(
            tickvals=angles,
            ticktext=labels,
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    )
)

# 顯示圖表
fig.show()
